In [ ]:
import cv2
import numpy as np
from sklearn.preprocessing import StandardScaler
import joblib
import mediapipe as mp

# 모델 불러오기
model = joblib.load('hand_gesture_model.joblib')

# Mediapipe Hands 모듈 초기화
mp_hands = mp.solutions.hands
hands = mp_hands.Hands()

# OpenCV 비디오 캡처 초기화
cap = cv2.VideoCapture(0)

# 데이터 표준화
scaler = StandardScaler()

# Ensure that the scaler is fitted before using it
scaler_fitted = False

while True:
    ret, frame = cap.read()
    if not ret:
        break

    # Mediapipe를 사용하여 손 인식
    results = hands.process(frame)

    if results.multi_hand_landmarks:
        # 손이 하나 이상 감지된 경우
        features_left = np.zeros((2, 21))
        features_right = np.zeros((2, 21))

        # 좌표 추출 및 저장
        for i, landmarks_per_hand in enumerate(results.multi_hand_landmarks):
            landmarks = np.array([[landmark.x, landmark.y] for landmark in landmarks_per_hand.landmark])
            if i == 0:
                features_left = landmarks.T  # 전치하여 (2, 21)로 만듦
            elif i == 1:
                features_right = landmarks.T  # 전치하여 (2, 21)로 만듦

        # 두 손의 데이터를 합칠 때, 2차원 배열로 만들어줌
        combined_features = np.concatenate([features_left, features_right], axis=0)

        # Ensure that the scaler is fitted before using it
        if not scaler_fitted:
            scaler.fit(combined_features)
            scaler_fitted = True


        # 데이터 표준화
        hand_data_standardized = scaler.transform(combined_features)

        # 데이터를 (None, 21, 4) 모양으로 재구성
        hand_data_standardized = hand_data_standardized.reshape(-1, 21, 4)

        # 예측 수행
        predicted_label = model.predict(np.array([hand_data_standardized]))
        
        # 분류 결과를 화면에 표시
        class_idx = np.argmax(predicted_label)
        if class_idx == 0:
            label = "o"
        elif class_idx == 1:
            label = "x"
        else:
            label = "a"
        cv2.putText(frame, label, (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

        # 예측 결과 출력
        print("Predicted label for current hand gesture:", predicted_label.tolist())

    cv2.imshow("Hand Tracking", frame)

    # 'q' 키를 누르면 종료
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# 비디오 캡처와 Mediapipe 해제
cap.release()
cv2.destroyAllWindows()
hands.close()

In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import StandardScaler

# MediaPipe Hands 모델 로드
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=2)

# TensorFlow SavedModel 형식으로 저장된 모델 로드
loaded_model = tf.keras.models.load_model("hand_gesture_model")

# 스케일러 초기화
scaler = StandardScaler()
scaler_fitted = False

# 비디오 캡처 시작
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # 손의 랜드마크 좌표 추출
    image = cv2.cvtColor(cv2.flip(frame, 1), cv2.COLOR_BGR2RGB)
    results = hands.process(image)

    if results.multi_hand_landmarks:
        # 손이 하나 이상 감지된 경우
        features_left = np.zeros((2, 21))
        features_right = np.zeros((2, 21))

        # 좌표 추출 및 저장
        for i, landmarks_per_hand in enumerate(results.multi_hand_landmarks):
            landmarks = np.array([[landmark.x, landmark.y] for landmark in landmarks_per_hand.landmark])
            if i == 0:
                features_left = landmarks.T 
            elif i == 1:
                features_right = landmarks.T  

        # 두 손의 데이터를 합칠 때, 2차원 배열로 만들어줌
        combined_features = np.concatenate([features_left, features_right], axis=0)

        # Ensure that the scaler is fitted before using it
        if not scaler_fitted:
            scaler.fit(combined_features)
            scaler_fitted = True

        # 모델 입력 형식에 맞게 데이터 스케일링
        scaled_features = scaler.transform(combined_features)


        # 모델의 입력 형태에 맞게 데이터 변환
        scaled_features = scaled_features.reshape(-1, 21, 4)

        # 모델에 입력하여 예측
        prediction = loaded_model.predict(scaled_features)

        
        # 분류 결과를 화면에 표시
        class_idx = np.argmax(prediction)
        if class_idx == 0:
            label = "o"
        elif class_idx == 1:
            label = "x"
        else:
            label = "a"
        cv2.putText(frame, label, (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

    cv2.imshow('Hand Classification', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# 비디오 캡처 종료
cap.release()
cv2.destroyAllWindows()
hands.close()

In [6]:
# 실시간 
# 랜드마크에서 최소 경계 사각형을 계산하고, 해당 사각형 내의 좌표만을 추출한 후, 이를 모델에 입력하여 예측 결과를 화면에 표시

import cv2
import mediapipe as mp
import numpy as np
import tensorflow as tf
from sklearn.preprocessing import StandardScaler

# MediaPipe Hands 모델 로드
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(static_image_mode=False, max_num_hands=2)

# TensorFlow SavedModel 형식으로 저장된 모델 로드
loaded_model = tf.keras.models.load_model("best_model.h11")

# 스케일러 초기화
scaler = StandardScaler()
scaler_fitted = False

# 최소 경계 사각형 계산 함수
def calculate_bounding_box(landmarks):
    x_values = [landmark[0] for landmark in landmarks]
    y_values = [landmark[1] for landmark in landmarks]
    min_x = min(x_values)
    max_x = max(x_values)
    min_y = min(y_values)
    max_y = max(y_values)
    return min_x, min_y, max_x, max_y

# 비디오 캡처 시작
cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    image = cv2.cvtColor(cv2.flip(frame, 1), cv2.COLOR_BGR2RGB)
    results = hands.process(image)

    if results.multi_hand_landmarks:
        for landmarks_per_hand in results.multi_hand_landmarks:
            landmarks = np.array([[landmark.x, landmark.y] for landmark in landmarks_per_hand.landmark])

            # 최소 경계 사각형 계산
            min_x, min_y, max_x, max_y = calculate_bounding_box(landmarks)

            # 최소 경계 사각형 안의 좌표 추출
            landmarks_in_bbox = []
            for landmark in landmarks:
                if min_x <= landmark[0] <= max_x and min_y <= landmark[1] <= max_y:
                    landmarks_in_bbox.append(landmark)

            landmarks_in_bbox = np.array(landmarks_in_bbox)

            # Ensure that the scaler is fitted before using it
            if not scaler_fitted:
                scaler.fit(landmarks_in_bbox)
                scaler_fitted = True

            # 모델 입력 형식에 맞게 데이터 스케일링
            scaled_features = scaler.transform(landmarks_in_bbox)

            # 모델의 입력 형태에 맞게 데이터 변환
            scaled_features = scaled_features.reshape(-1, 21, 4)

            # 모델에 입력하여 예측
            prediction = loaded_model.predict(scaled_features)

            # 분류 결과를 화면에 표시
            class_idx = np.argmax(prediction)
            if class_idx == 0:
                label = "o"
            elif class_idx == 1:
                label = "x"
            else:
                label = "a"
            cv2.putText(frame, label, (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)

    cv2.imshow('Hand Classification', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# 비디오 캡처 종료
cap.release()
cv2.destroyAllWindows()
hands.close()


OSError: Unable to synchronously open file (file signature not found)

In [2]:
import cv2
import mediapipe as mp
import numpy as np
import tensorflow as tf

# 손 모델 로딩
mp_hands = mp.solutions.hands
hands = mp_hands.Hands()

model = tf.keras.models.load_model("hand_gesture_model")

# 웹캠 열기
cap = cv2.VideoCapture(0)

# 반복 실행
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # OpenCV를 사용하여 BGR 이미지를 RGB로 변환
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Mediapipe를 사용하여 양손 감지
    results = hands.process(rgb_frame)

    if results.multi_hand_landmarks and len(results.multi_hand_landmarks) >= 2:
        # 좌표를 저장할 리스트 초기화
        all_x_coords_left = []
        all_y_coords_left = []
        all_x_coords_right = []
        all_y_coords_right = []
        
        for hand_landmarks in results.multi_hand_landmarks:
            # 감지된 손의 landmark를 사용하여 좌표 저장
            landmarks = hand_landmarks.landmark
            x_coords = [landmark.x for landmark in landmarks]
            y_coords = [landmark.y for landmark in landmarks]
            
            if landmarks[mp_hands.HandLandmark.WRIST].x < 0.5:
                all_x_coords_left.extend(x_coords)
                all_y_coords_left.extend(y_coords)
            else:
                all_x_coords_right.extend(x_coords)
                all_y_coords_right.extend(y_coords)

        if all_x_coords_left and all_y_coords_left and all_x_coords_right and all_y_coords_right:
            # Calculate bounding box encompassing both hands
            x_min_left = min(all_x_coords_left)
            y_min_left = min(all_y_coords_left)
            x_max_left = max(all_x_coords_left)
            y_max_left = max(all_y_coords_left)
            box_width_left = x_max_left - x_min_left
            box_height_left = y_max_left - y_min_left

            x_min_right = min(all_x_coords_right)
            y_min_right = min(all_y_coords_right)
            x_max_right = max(all_x_coords_right)
            y_max_right = max(all_y_coords_right)
            box_width_right = x_max_right - x_min_right
            box_height_right = y_max_right - y_min_right

            # Normalize coordinates by box width and height
            x_coords_normalized_left = [(x - x_min_left) * 100 / box_width_left for x in all_x_coords_left]
            y_coords_normalized_left = [(y - y_min_left) * 100 / box_height_left for y in all_y_coords_left]
            x_coords_normalized_right = [(x - x_min_right) * 100 / box_width_right for x in all_x_coords_right]
            y_coords_normalized_right = [(y - y_min_right) * 100 / box_height_right for y in all_y_coords_right]

            # 모델에 입력할 데이터 구성
            input_data_left = np.array([[x, y] for x, y in zip(x_coords_normalized_left, y_coords_normalized_left)])
            input_data_right = np.array([[x, y] for x, y in zip(x_coords_normalized_right, y_coords_normalized_right)])

            # 모델 예측
            pre1 = model.predict(np.expand_dims(input_data_left, axis=0))
            pre2 = model.predict(np.expand_dims(input_data_right, axis=0))

            # 분류 결과를 화면에 표시
            class_idx1 = np.argmax(pre1)
            class_idx2 = np.argmax(pre2)

            label1 = str(class_idx1)
            label2 = str(class_idx2)

            cv2.putText(frame, f"Left Hand: {label1}", (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2, cv2.LINE_AA)
            cv2.putText(frame, f"Right Hand: {label2}", (10, 100), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2, cv2.LINE_AA)

    cv2.imshow('Hand Classification', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# 사용된 자원 해제
cap.release()
cv2.destroyAllWindows()

ValueError: in user code:

    File "C:\Users\user\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py", line 2440, in predict_function  *
        return step_function(self, iterator)
    File "C:\Users\user\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py", line 2425, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "C:\Users\user\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py", line 2413, in run_step  **
        outputs = model.predict_step(data)
    File "C:\Users\user\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py", line 2381, in predict_step
        return self(x, training=False)
    File "C:\Users\user\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\utils\traceback_utils.py", line 70, in error_handler
        raise e.with_traceback(filtered_tb) from None
   

In [7]:
import cv2
import mediapipe as mp
import numpy as np
import tensorflow as tf

# 손 모델 로딩
mp_hands = mp.solutions.hands
hands = mp_hands.Hands()

#model_left = tf.keras.models.load_model("left_hand_gesture_model")  # 왼손 모델 로드
#model_right = tf.keras.models.load_model("right_hand_gesture_model")  # 오른손 모델 로드

model_left = tf.keras.models.load_model("best_model.h11")  # 왼손 모델 로드
model_right = tf.keras.models.load_model("best_model.h11")  # 오른손 모델 로드

# 웹캠 열기
cap = cv2.VideoCapture(0)

# 반복 실행
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # OpenCV를 사용하여 BGR 이미지를 RGB로 변환
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Mediapipe를 사용하여 양손 감지
    results = hands.process(rgb_frame)

    if results.multi_hand_landmarks and len(results.multi_hand_landmarks) >= 2:
        # 좌표를 저장할 리스트 초기화
        all_x_coords_left = []
        all_y_coords_left = []
        all_x_coords_right = []
        all_y_coords_right = []
        
        for hand_landmarks in results.multi_hand_landmarks:
            # 감지된 손의 landmark를 사용하여 좌표 저장
            landmarks = hand_landmarks.landmark
            x_coords = [landmark.x for landmark in landmarks]
            y_coords = [landmark.y for landmark in landmarks]
            
            if landmarks[mp_hands.HandLandmark.WRIST].x < 0.5:
                all_x_coords_left.extend(x_coords)
                all_y_coords_left.extend(y_coords)
            else:
                all_x_coords_right.extend(x_coords)
                all_y_coords_right.extend(y_coords)

        if all_x_coords_left and all_y_coords_left and all_x_coords_right and all_y_coords_right:
            # Calculate bounding box encompassing both hands
            x_min_left = min(all_x_coords_left)
            y_min_left = min(all_y_coords_left)
            x_max_left = max(all_x_coords_left)
            y_max_left = max(all_y_coords_left)
            box_width_left = x_max_left - x_min_left
            box_height_left = y_max_left - y_min_left

            x_min_right = min(all_x_coords_right)
            y_min_right = min(all_y_coords_right)
            x_max_right = max(all_x_coords_right)
            y_max_right = max(all_y_coords_right)
            box_width_right = x_max_right - x_min_right
            box_height_right = y_max_right - y_min_right

            # Normalize coordinates by box width and height
            x_coords_normalized_left = [(x - x_min_left) * 100 / box_width_left for x in all_x_coords_left]
            y_coords_normalized_left = [(y - y_min_left) * 100 / box_height_left for y in all_y_coords_left]
            x_coords_normalized_right = [(x - x_min_right) * 100 / box_width_right for x in all_x_coords_right]
            y_coords_normalized_right = [(y - y_min_right) * 100 / box_height_right for y in all_y_coords_right]

            # 모델에 입력할 데이터 구성
            input_data_left = np.array([[x, y] for x, y in zip(x_coords_normalized_left, y_coords_normalized_left)])
            input_data_right = np.array([[x, y] for x, y in zip(x_coords_normalized_right, y_coords_normalized_right)])

            # 모델 예측
            pre1 = model_left.predict(np.expand_dims(input_data_left, axis=0))
            pre2 = model_right.predict(np.expand_dims(input_data_right, axis=0))

            # 양손 모두가 'rock'인지 확인
            if np.argmax(pre1) == 1 and np.argmax(pre2) == 1:
                print("Both hands are rock")
                result = "rock"
            # 양손 모두가 'paper'인지 확인
            elif np.argmax(pre1) == 0 and np.argmax(pre2) == 0:
                print("Both hands are paper")
                result = "paper"
            else:
                print("Other gestures")
                result = "etc"

            # 결과를 화면에 표시
            cv2.putText(frame, f"Result: {result}", (10, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2, cv2.LINE_AA)

    cv2.imshow('Hand Classification', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# 사용된 자원 해제
cap.release()
cv2.destroyAllWindows()




1/1 [==============================] - 1s 518ms/step
Other gestures
1/1 [==============================] - 0s 30ms/step
Other gestures
1/1 [==============================] - 0s 27ms/step
Other gestures
1/1 [==============================] - 0s 29ms/step
Other gestures
1/1 [==============================] - 0s 28ms/step
Other gestures
1/1 [==============================] - 0s 30ms/step
Other gestures
1/1 [==============================] - 0s 22ms/step
Other gestures
1/1 [==============================] - 0s 24ms/step
Other gestures
1/1 [==============================] - 0s 26ms/step
Other gestures
1/1 [==============================] - 0s 23ms/step


InvalidArgumentError: Graph execution error:

Detected at node sequential_6/dense_36/Relu defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "C:\Users\user\AppData\Local\Programs\Python\Python311\Lib\site-packages\ipykernel_launcher.py", line 17, in <module>

  File "C:\Users\user\AppData\Local\Programs\Python\Python311\Lib\site-packages\traitlets\config\application.py", line 1077, in launch_instance

  File "C:\Users\user\AppData\Local\Programs\Python\Python311\Lib\site-packages\ipykernel\kernelapp.py", line 701, in start

  File "C:\Users\user\AppData\Local\Programs\Python\Python311\Lib\site-packages\tornado\platform\asyncio.py", line 205, in start

  File "C:\Users\user\AppData\Local\Programs\Python\Python311\Lib\asyncio\windows_events.py", line 321, in run_forever

  File "C:\Users\user\AppData\Local\Programs\Python\Python311\Lib\asyncio\base_events.py", line 607, in run_forever

  Fi